In [3]:
import pandas as pd

import pickle

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, log_loss, roc_curve, roc_auc_score, auc, RocCurveDisplay
from sklearn.linear_model import LogisticRegression

import xlearn as xl

In [2]:
my_feathers = ['zone_id', 'banner_id', 'campaign_clicks', 'os_id', 'country_id', 'oaid_hash']

Загрузим данные с моего гугл диска, так как скачать напрямую по ссылке не получилось.

In [3]:
%%time
ssss = '''

from google.colab import drive

drive.mount('/content/gdrive')


!cp -r "/content/gdrive/My Drive/recsys/HW1_data.csv" "/content/data.csv"
!cp -r "/content/gdrive/My Drive/recsys/control_data_flatten.pkl" "/content/control_data_flatten.pkl"
!cp -r "/content/gdrive/My Drive/recsys/y_test.pkl" "/content/y_test.pkl"
!cp -r "/content/gdrive/My Drive/recsys/data_flatten.pkl" "/content/data_flatten.pkl"
!cp -r "/content/gdrive/My Drive/recsys/control_data.pkl" "/content/control_data.pkl"
!cp -r "/content/gdrive/My Drive/recsys/x_test.pkl" "/content/x_test.pkl"
!cp -r "/content/gdrive/My Drive/recsys/y_train.pkl" "/content/y_train.pkl"
!cp -r "/content/gdrive/My Drive/recsys/data_after_celar.csv" "/content/data_after_celar.csv"
!cp -r "/content/gdrive/My Drive/recsys/x_train.pkl" "/content/x_train.pkl"
'''

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.15 µs


In [4]:
%%time
data = pd.read_csv('data.csv')
print(data.shape)
print(data.head())

(15821472, 17)
                    date_time  zone_id  banner_id            oaid_hash  \
0  2021-09-27 00:01:30.000000        0          0  5664530014561852622   
1  2021-09-26 22:54:49.000000        1          1  5186611064559013950   
2  2021-09-26 23:57:20.000000        2          2  2215519569292448030   
3  2021-09-27 00:04:30.000000        3          3  6262169206735077204   
4  2021-09-27 00:06:21.000000        4          4  4778985830203613115   

   campaign_clicks  os_id  country_id  banner_id0  rate0        g0  \
0                0      0           0        1240  0.067  0.035016   
1                0      0           1           1  0.002  0.054298   
2                3      0           0           2  0.014  0.014096   
3                0      1           1           3  0.012  0.015232   
4                0      1           0           4  0.019  0.051265   

   coeff_sum0  banner_id1  rate1        g1  coeff_sum1  impressions  clicks  
0   -7.268846           0  0.010  0.04951

Удалим колонки, которые нам не нужны.

In [5]:
%%time
data.drop(['banner_id0', 'banner_id1', 'rate0', 'rate1', 'g0', 'g1', 'coeff_sum0', 'coeff_sum1'],axis = 1, inplace = True)

CPU times: user 770 ms, sys: 460 ms, total: 1.23 s
Wall time: 2.67 s


Проанализируем информацию в стобцах.


In [6]:
%%time
def analysis(data: pd.DataFrame):
    for colomn in data.columns:
        uniq = data[colomn].unique()
        colomn_uniq_size = uniq.size

        print(colomn)
        print('size', colomn_uniq_size)
        print('min, max', uniq.min(), uniq.max())
        if colomn_uniq_size < 50:
            print(uniq)

    pass

small_data = data.head()
#analysis(small_data)
analysis(data)

date_time
size 604712
min, max 2021-09-01 00:02:49.000000 2021-10-02 23:59:59.000000
zone_id
size 3444
min, max 0 3443
banner_id
size 1633
min, max 0 1632
oaid_hash
size 6510316
min, max 1116910879938 9223371107563590722
campaign_clicks
size 822
min, max 0 829
os_id
size 11
min, max 0 10
[ 0  1  2  3  4  5  6  7  8  9 10]
country_id
size 17
min, max 0 16
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
impressions
size 1
min, max 1 1
[1]
clicks
size 2
min, max 0 1
[1 0]
CPU times: user 8.68 s, sys: 1.13 s, total: 9.81 s
Wall time: 16.1 s


Объеденим в каждой колонке все поля, где различных значений меньше 10

In [7]:
data.where(data.apply(lambda x: x.map(x.value_counts())) >= 10, -1, inplace=True)

In [ ]:
analysis(data)

во всех полях кроме даты малое количество различных значений, поэтому будем использовать ван хот енкодинг. Так же можно удалить столбец "impressions" так как во всех строчках данных одно и то же значение.
Время использовать не будем, так как данные всего за месяц, если не получиться ,то будем добавлять признаки на подоби и день недели/час дня.

In [8]:
%%time
data.drop(['impressions'],axis = 1, inplace = True)



CPU times: user 375 ms, sys: 140 ms, total: 515 ms
Wall time: 512 ms


Перенумеруем все поля в колонках

In [9]:
data[my_feathers] = data[my_feathers].apply(lambda x: pd.factorize(x)[0])


In [10]:
data.to_csv('data_after_celar.csv', index=False)

Сохраним для простоты, так как файл слишком большой


In [11]:
%%time

data = pd.read_csv('data_after_celar.csv')#.astype(pd.SparseDtype(int, fill_value=0))
print(data.shape)
print(data.head())

(15821472, 8)
                    date_time  zone_id  banner_id  oaid_hash  campaign_clicks  \
0  2021-09-27 00:01:30.000000        0          0          0                0   
1  2021-09-26 22:54:49.000000        1          1          0                0   
2  2021-09-26 23:57:20.000000        2          2          1                1   
3  2021-09-27 00:04:30.000000        3          3          0                0   
4  2021-09-27 00:06:21.000000        4          4          0                0   

   os_id  country_id  clicks  
0      0           0       1  
1      0           1       1  
2      0           0       1  
3      1           1       1  
4      1           0       1  
CPU times: user 14.5 s, sys: 4.84 s, total: 19.4 s
Wall time: 27.5 s


In [13]:
max_value = 0
for colomn in my_feathers:
    data[colomn].unique()
    #max_value = max(max_value, data[colomn].max() + 1)
    max_value = max(max_value, data[colomn].unique().size + 1)
print(max_value)

217990


Отделим последний день от набора данных, так как по условию задания нужно сравнить метрики на последнем дне. и нет смыла на нем обучаться.

In [14]:
%%time
mask = data['date_time'] >= '2021-10-02'
print('mask ready')
control_data = data[mask]
print('mask ready')
control_data.reset_index(drop=True, inplace = True)
print('control ready')
mask = data['date_time'] < '2021-10-02'
print('mask ready')
data = data[mask]
print('mask ready')
data.reset_index(drop=True, inplace = True)
print('data ready')

control_data.drop(['date_time'], axis = 1, inplace = True)
print('control without date ready')
data.drop(['date_time'], axis = 1, inplace = True)
print('data without date ready')

mask ready
mask ready
control ready
mask ready
mask ready
data ready
control without date ready
data without date ready
CPU times: user 3.21 s, sys: 496 ms, total: 3.71 s
Wall time: 4.2 s


In [15]:

control_data.to_csv('control_data_flatten.csv', index=False)
data.to_csv('data_flatten.csv', index=False)

y_control = control_data['clicks']
y_control.to_csv('y_control.csv', index=False)

In [3]:
%%time


control_data = pd.read_csv('control_data_flatten.csv')#.astype(pd.SparseDtype(int, fill_value=0))
print(control_data.shape)
data = pd.read_csv('data_flatten.csv')#.astype(pd.SparseDtype(int, fill_value=0))
print(data.shape)

(2119894, 7)
(13701578, 7)
CPU times: user 4.94 s, sys: 939 ms, total: 5.88 s
Wall time: 6.42 s


Отделим разделим оставшиеся даные на тест и траин.

In [4]:
test = data.sample(frac=0.2)
train = data.drop(test.index)
del data
train = train.sample(frac=1)

In [5]:
test.to_csv('test.csv', index=False)
train.to_csv('train.csv', index=False)

y_test = test['clicks']
y_test.to_csv('y_test.csv', index=False)

y_train = train['clicks']
y_train.to_csv('y_train.csv', index=False)

In [6]:
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')

https://github.com/ycjuan/libffm
Привидем данные к такому виду, чтобы использовать ffm в xlearn.

max_value - максимальное колиичество уникальных значений в колонке

In [8]:
max_value = 217990

In [9]:
%%time
def feature_engineering(file_name, data):
    with open(file_name, "w") as f:
        for index, row in data.iterrows():
            if index % 200000 == 0:
                print(index)
            result_row = str(row['clicks'])
            for i, feather in enumerate(my_feathers):
                s = ' ' + str(i) + ':' + str(i * max_value + row[feather]) + ':1'
                result_row += s
            result_row += '\n'
            f.write(result_row)
        
            
feature_engineering('test.txt', test)
feature_engineering('train.txt', train)
feature_engineering('control_data.txt', control_data)

0
200000
400000
600000
800000
1000000
1200000
1400000
1600000
1800000
2000000
2200000
2400000
2600000
0
200000
400000
600000
800000
1000000
1200000
1400000
1600000
1800000
2000000
2200000
2400000
2600000
2800000
3000000
3200000
3400000
3600000
3800000
4000000
4200000
4400000
4600000
4800000
5000000
5200000
5400000
5600000
5800000
6000000
6200000
6400000
6600000
6800000
7000000
7200000
7400000
7600000
7800000
8000000
8200000
8400000
8600000
8800000
9000000
9200000
9400000
9600000
9800000
10000000
10200000
10400000
10600000
10800000
0
200000
400000
600000
800000
1000000
1200000
1400000
1600000
1800000
2000000
CPU times: user 22min 12s, sys: 3.14 s, total: 22min 16s
Wall time: 22min 16s


In [8]:
k_value = [1, 2, 4, 8]
l_value = [0.00001, 0.0001, 0.001]

In [ ]:
%%time
y_test = pd.read_csv('y_test.csv')

for k in k_value:
    for l in l_value:
        print('k : ', k, ' lambda : ', l)
        ffm_model = xl.create_ffm()
        ffm_model.setTrain("train.txt")
        ffm_model.setTest("test.txt")
        param = {'task':'binary', 'lr': 0.1, 'lambda': l, 'k': k, 'metric': 'auc'}

        ffm_model.fit(param, './model.out')
        ffm_model.setSigmoid()
        ffm_model.predict('./model.out', './output.txt')

        with open('output.txt', 'r') as f:
            y_pred_proba = np.array(list(map(float, filter(lambda s: len(s) > 0, f.read().split('\n')))))
        roc_auc_metric = roc_auc_score(y_test, y_pred_proba)
        log_loss_metric = log_loss(y_test, y_pred_proba)
        print('roc_auc : ', roc_auc_metric, ' log_loss : ',log_loss_metric)
        
        
        

k :  1  lambda :  0.001
----------------------------------------------------------------------------------------------
           _
          | |
     __  _| |     ___  __ _ _ __ _ __
     \ \/ / |    / _ \/ _` | '__| '_ \ 
      >  <| |___|  __/ (_| | |  | | | |
     /_/\_\_____/\___|\__,_|_|  |_| |_|

        xLearn   -- 0.40 Version --
----------------------------------------------------------------------------------------------

[ WARNING    ] Validation file not found, xLearn has already disable early-stopping.
[ WARNING    ] Validation file not found, xLearn has already disable (-x auc) option.
[------------] xLearn uses 4 threads for training task.
[ ACTION     ] Read Problem ...
[------------] First check if the text file has been already converted to binary format.
[------------] Binary file (train.txt.bin) found. Skip converting text to binary.
[------------] Number of Feature: 1307939
[------------] Number of Field: 6
[------------] Time cost for reading problem: 8.11 (sec)


[------------] Binary file (train.txt.bin) found. Skip converting text to binary.
[------------] Number of Feature: 1307939
[------------] Number of Field: 6
[------------] Time cost for reading problem: 33.30 (sec)
[ ACTION     ] Initialize model ...
[------------] Model size: 249.47 MB
[------------] Time cost for model initial: 0.38 (sec)
[ ACTION     ] Start to train ...
[------------] Epoch      Train log_loss     Time cost (sec)
[   10%      ]     1            0.106319                4.64
[   20%      ]     2            0.104869                4.91
[   30%      ]     3            0.104111                4.67
[   40%      ]     4            0.103469                4.71
[   50%      ]     5            0.102923                4.95
[   60%      ]     6            0.102406                4.58
[   70%      ]     7            0.101981                4.99
[   80%      ]     8            0.101571                5.03
[   90%      ]     9            0.101178                4.42
[  100%     

Перебор параметров в одной сессии осуществляется плохо, но я выбрал параметры при которых работает хорошо

In [5]:
#k :  4  lambda :  0.0001
k = 4
l = 0.0001

In [6]:
y_control = pd.read_csv('y_control.csv')

print('k : ', k, ' lambda : ', l)
ffm_model = xl.create_ffm()
ffm_model.setTrain("train.txt")
ffm_model.setTest("control_data.txt")
param = {'task':'binary', 'lr': 0.1, 'lambda': l, 'k': k, 'metric': 'auc'}

ffm_model.fit(param, './model.out')
ffm_model.setSigmoid()
ffm_model.predict('./model.out', './output.txt')

with open('output.txt', 'r') as f:
    y_pred_proba = np.array(list(map(float, filter(lambda s: len(s) > 0, f.read().split('\n')))))
roc_auc_metric = roc_auc_score(y_control, y_pred_proba)
log_loss_metric = log_loss(y_control, y_pred_proba)
print('roc_auc : ', roc_auc_metric, ' log_loss : ',log_loss_metric)

k :  4  lambda :  0.0001
----------------------------------------------------------------------------------------------
           _
          | |
     __  _| |     ___  __ _ _ __ _ __
     \ \/ / |    / _ \/ _` | '__| '_ \ 
      >  <| |___|  __/ (_| | |  | | | |
     /_/\_\_____/\___|\__,_|_|  |_| |_|

        xLearn   -- 0.40 Version --
----------------------------------------------------------------------------------------------

[ WARNING    ] Validation file not found, xLearn has already disable early-stopping.
[ WARNING    ] Validation file not found, xLearn has already disable (-x auc) option.
[------------] xLearn uses 4 threads for training task.
[ ACTION     ] Read Problem ...
[------------] First check if the text file has been already converted to binary format.
[------------] Binary file (train.txt.bin) found. Skip converting text to binary.
[------------] Number of Feature: 1307939
[------------] Number of Field: 6
[------------] Time cost for reading problem: 7.78 (sec)

В прошлой домашней работе AUC:0.76, а в этой 0.81.

В прошлой домашней работе logloss:0.14, а в этой 0.125
